In [ ]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
import spacy
import wordcloud
import os 
import sys
pd.options.mode.chained_assignment = None 

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
nltk.download('wordnet')
nltk.download('punkt')

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
!python -m spacy download en_core_web_md
import en_core_web_md
text_to_nlp = en_core_web_md.load()

In [ ]:
data_file  = 'yelp_final.csv'

!wget https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%201%20-%205/Session%203%20-%20NLP/yelp_final.csv


In [ ]:
yelp_full = pd.read_csv(data_file)
yelp_full.head()

In [ ]:
needed_columns = [] 
yelp = yelp_full[needed_columns]
yelp.head()

In [ ]:
needed_columns = ['stars','text']
yelp = yelp_full[needed_columns]
yelp.head()

In [ ]:
num_stars =  1

for t in yelp[yelp['stars'] == num_stars]['text'].head(20).values:
    print (t) 

In [ ]:
num_stars =  1
this_star_text = ''
for t in yelp[yelp['stars'] == num_stars]['text'].values: 
    this_star_text += t + ' '
    
wordcloud = WordCloud()    
wordcloud.generate_from_text(this_star_text)
plt.figure(figsize=(14,7))
plt.imshow(wordcloud, interpolation='bilinear')

In [ ]:
def is_good_review(num_stars):
    if num_stars > 3: 
        return True
    else:
        return False

yelp['is_good_review'] = yelp['stars'].apply(is_good_review)
yelp.head()

In [ ]:
example_text = "All the people I spoke to were super nice and very welcoming." #@param {type:"string"}
tokens = word_tokenize(example_text)
tokens

In [ ]:
example_word = "not" 
if example_word.lower() in STOP_WORDS:
  print ('"' + example_word + '" is a stop word.')
else:
  print ('"' + example_word + '" is NOT a stop word.')

In [ ]:
X_text = yelp['text']
y = yelp['is_good_review']

In [ ]:
def tokenize(text):
    clean_tokens = []
    for token in text_to_nlp(text):
        if (not token.is_stop) & (token.lemma_ != '-PRON-') & (not token.is_punct): # -PRON- is a special all inclusive "lemma" spaCy uses for any pronoun, we want to exclude these 
            clean_tokens.append(token.lemma_)
    return clean_tokens

In [ ]:
bow_transformer = CountVectorizer(analyzer=tokenize, max_features=800).fit(X_text)

In [ ]:
bow_transformer.vocabulary_

In [ ]:
len(bow_transformer.vocabulary_)

In [ ]:
X = bow_transformer.transform(X_text)

In [ ]:
pd.DataFrame(X.toarray())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

In [ ]:
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print (accuracy)

In [ ]:
example_review = "This restaurant sucks" 
prediction = logistic_model.predict(bow_transformer.transform([example_review]))

if prediction:
  print ("This was a GOOD review!")
else:
  print ("This was a BAD review!")



In [ ]:
stakeholders = '' 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()

In [ ]:
nb_model.fit(X_train, y_train)
nb_preds = nb_model.predict(X_test) 
accuracy = accuracy_score(y_test, y_pred)
print ("The accuracy of the model is " + str(accuracy*100) + "%")

In [ ]:
text_to_nlp = en_core_web_md.load() 

In [ ]:
doc = text_to_nlp("I like apples and cherries and peaches and pie")
token = doc[2] 
print (token)
print (len(token))

In [ ]:
print ('Vector for: ', token)
print (token.vector) 

In [ ]:
doc = text_to_nlp(u"guitar and piano")
word1 = doc[0]
word2 = doc[2]
word1.similarity(word2)

In [ ]:
def tokenize_vecs(text):
    clean_tokens = []
    for token in text_to_nlp(text):
        if (not token.is_stop) & (token.lemma_ != '-PRON-') & (not token.is_punct): 
            clean_tokens.append(token)
    return clean_tokens

In [ ]:
X_word2vec = []
for text in X_text:
  review = tokenize_vecs(text) 
  review_vec = [0]*300
  for word in review:
    review_vec += word.vector
  review_vec = review_vec / len(review)
  X_word2vec.append(review_vec)

X_word2vec = np.array(X_word2vec)


In [ ]:
w2v_model = LogisticRegression()

X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(X_word2vec, y, test_size=0.2, random_state=101)

w2v_model.fit(X_train_word2vec, y_train_word2vec)

w2v_preds = w2v_model.predict(X_test_word2vec) 
accuracy = accuracy_score(y_test_word2vec, w2v_preds)
print (accuracy)

In [ ]:
vocab_dict = dict() 

for word in bow_transformer.vocabulary_:
    vocab_dict[word] = text_to_nlp(word).vector 

for word, vec in vocab_dict.items(): 
  print ('Word: {}. Vector length: {}'.format(word, len(vec)))

print()
print ('{} words in our dictionary'.format(len(vocab_dict)))

In [ ]:
v0 = [2,3,1]
v1 = [2,4,1]

In [ ]:
import numpy as np 
import plotly.graph_objs as go

def vector_plot(tvects,is_vect=True,orig=[0,0,0]):
    """Plot vectors using plotly"""

    if is_vect:
        if not hasattr(orig[0],"__iter__"):
            coords = [[orig,np.sum([orig,v],axis=0)] for v in tvects]
        else:
            coords = [[o,np.sum([o,v],axis=0)] for o,v in zip(orig,tvects)]
    else:
        coords = tvects

    data = []
    for i,c in enumerate(coords):
        X1, Y1, Z1 = zip(c[0])
        X2, Y2, Z2 = zip(c[1])
        vector = go.Scatter3d(x = [X1[0],X2[0]],
                              y = [Y1[0],Y2[0]],
                              z = [Z1[0],Z2[0]],
                              marker = dict(size = [0,5],
                                            color = ['blue'],
                                            line=dict(width=5,
                                                      color='DarkSlateGrey')),
                              name = 'Vector'+str(i+1))
        data.append(vector)

    layout = go.Layout(
             margin = dict(l = 4,
                           r = 4,
                           b = 4,
                           t = 4)
                  )
    fig = go.Figure(data=data,layout=layout)
    fig.show()


vector_plot([v0,v1])

In [ ]:
def vector_cosine_similarity(vec1,vec2):

  numerator = 0
  for i in range(len(vec1)):
    numerator += vec1[i]*vec2[i]
  mag1 = (sum(elem**2 for elem in vec1))**0.5
  mag2 = (sum(elem**2 for elem in vec2))**0.5
  similarity = numerator/(mag1*mag2)
  return similarity

print(vector_cosine_similarity(v0,v1))

In [ ]:
def word_similarity(word1, word2):
  
  try:
    vec1 = vocab_dict[word1]
    vec2 = vocab_dict[word2]
    return vector_cosine_similarity(vec1,vec2)

  except KeyError:
    print ('Word not in dictionary')

print(word_similarity('burger','steak'))

In [ ]:
def find_nearest_neighbor(word):
  try:
    vec = vocab_dict[word]
    find_most_similar(vec)
  except KeyError:
    print ('Word not in dictionary')

def find_most_similar(start_vec):
 
  similarity_series = pd.Series(np.nan, index = vocab_dict.keys())
  for word, vec in vocab_dict.items():
    similarity_series[word] = vector_cosine_similarity(start_vec, vec)
  similarity_series = similarity_series[similarity_series.notna()]  
  five_most_similar = similarity_series.sort_values().tail()
  print (five_most_similar) 

find_nearest_neighbor('bagel')

In [ ]:
def find_analogy(word_a1, word_a2, word_b1):
  
  a1_vec = vocab_dict[word_a1]
  a2_vec = vocab_dict[word_a2]
  b1_vec = vocab_dict[word_b1]
  find_most_similar(b1_vec - a1_vec + a2_vec)

find_analogy('breakfast','bagel','lunch')